In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

import warnings
warnings.filterwarnings('ignore')

In [2]:
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Activation, Conv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D, InputLayer, Flatten, BatchNormalization

from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras_vggface.vggface import VGGFace
from keras.applications.vgg16 import preprocess_input
from keras.optimizers import RMSprop, Adam
from keras.preprocessing.image import load_img, ImageDataGenerator

import keras_metrics
import pandas
import ast
import numpy as np


Using TensorFlow backend.


In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [4]:
train_set = ".\\dataset\\NUOVO\\train"
validation_set = ".\\dataset\\NUOVO\\val"
test_set = ".\\dataset\\NUOVO\\test"

In [5]:
train_df = pandas.read_csv(".\\dataset\\NUOVO\\train\\output.csv")
valid_df = pandas.read_csv(".\\dataset\\NUOVO\\val\\output.csv")
test_df = pandas.read_csv(".\\dataset\\NUOVO\\test\\output.csv")

In [6]:
def random_crop(img, random_crop_size):
    # Note: image_data_format is 'channel_last'
    assert img.shape[2] == 3
    height, width = img.shape[0], img.shape[1]
    dy, dx = random_crop_size
    x = np.random.randint(0, width - dx + 1)
    y = np.random.randint(0, height - dy + 1)
    return img[y:(y+dy), x:(x+dx), :]

def crop_generator(batches, crop_length):
    """Take as input a Keras ImageGen (Iterator) and generate random
    crops from the image batches generated by the original iterator.
    """
    while True:
        batch_x, batch_y = next(batches)
        batch_crops = np.zeros((batch_x.shape[0], crop_length, crop_length, 3))
        for i in range(batch_x.shape[0]):
            batch_crops[i] = random_crop(batch_x[i], (crop_length, crop_length))
        yield (batch_crops, batch_y)

In [7]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    brightness_range=[0.4,1.0])

train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory='.\\dataset\\NUOVO\\train',
        x_col="image_id",
        y_col=["x_1","y_1","width","height"],
        class_mode="raw",
        shuffle=True,
        target_size=(224, 224),
        batch_size=64)

valid_generator = train_datagen.flow_from_dataframe(
        dataframe=valid_df,
        directory='.\\dataset\\NUOVO\\val',
        x_col="image_id",
        y_col=["x_1","y_1","width","height"],
        class_mode="raw",
        shuffle=True,
        target_size=(224, 224),
        batch_size=64)

train_crops = train_generator
val_crops = valid_generator

Found 164105 validated image filenames.
Found 18234 validated image filenames.


In [8]:
step_size_train = train_generator.samples/train_generator.batch_size
step_size_valid = valid_generator.samples/valid_generator.batch_size

In [9]:
model = Sequential()
model.add(InputLayer(input_shape=(224,224,3)))
model.add(Conv2D(filters=32, kernel_size=(3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=2))

model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=2))

model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=64, kernel_size=(1,1), activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=2))

model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(1,1), activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=2))

model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(1,1), activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(1,1), activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=2))
          
model.add(Conv2D(filters=1024, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(1,1), activation="relu"))
model.add(Conv2D(filters=1024, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(1,1), activation="relu"))
model.add(Conv2D(filters=1024, kernel_size=(3,3), padding="same", activation="relu"))

model.add(Conv2D(filters=4, kernel_size=(1,1)))
#model.add(GlobalAveragePooling2D())
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(100, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(100, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(4))

In [10]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 32)      896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 224, 224, 32)      128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 64)      18496     
_________________________________________________________________
batch_normalization_2 (Batch (None, 112, 112, 64)      256       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 56, 56, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 56, 56, 128)      

In [11]:
model.compile(optimizer=Adam(), loss='mse', metrics=[keras_metrics.precision()])

tracking <tf.Variable 'Variable:0' shape=() dtype=int32> tp
tracking <tf.Variable 'Variable_1:0' shape=() dtype=int32> fp


In [12]:
model.evaluate_generator(val_crops, steps=step_size_valid, use_multiprocessing=False, verbose=1)


285/284 [==============================] - 25s 89ms/step


[56953.02734375, 0.0]

In [13]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=12, verbose=1, mode='min')
mcp_save = ModelCheckpoint('yolf.h5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4, verbose=1, mode='min')

history = model.fit_generator(generator=train_crops, epochs=5, steps_per_epoch=step_size_train, validation_data=val_crops, validation_steps=step_size_valid, verbose=1, callbacks=[mcp_save, earlyStopping, reduce_lr_loss])

Epoch 1/5
2565/2564 [==============================] - 490s 191ms/step - loss: 17544.0402 - precision: 55.5013 - val_loss: 21756.5117 - val_precision: 169.7817
Epoch 2/5
2565/2564 [==============================] - 486s 190ms/step - loss: 15835.2101 - precision: 52.3492 - val_loss: 5049.6841 - val_precision: 184.8901
Epoch 3/5
2565/2564 [==============================] - 487s 190ms/step - loss: 15245.4388 - precision: 54.5572 - val_loss: 19425.7617 - val_precision: 174.2422
Epoch 4/5
2565/2564 [==============================] - 493s 192ms/step - loss: 14645.5572 - precision: 53.9407 - val_loss: 8159.5503 - val_precision: 186.9954
Epoch 5/5
2565/2564 [==============================] - 486s 190ms/step - loss: 14135.6020 - precision: 54.0284 - val_loss: 13851.8926 - val_precision: 205.5939


In [ ]:
print(history.history)

In [ ]:
acc = history.history['precision']
val_acc = history.history['val_precision']

rec = history.history['recall']
val_rec = history.history['val_recall']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(3, 1, 1)
plt.plot(acc, label='Training Precision')
plt.plot(val_acc, label='Validation Precision')
plt.legend(loc='lower right')
plt.ylabel('Precision')
plt.ylim([min(plt.ylim()),200])
plt.title('Training and Validation Precision')

plt.subplot(3, 1, 2)
plt.plot(rec, label='Training Recall')
plt.plot(val_rec, label='Validation Recall')
plt.legend(loc='lower right')
plt.ylabel('Recall')
plt.ylim([min(plt.ylim()),200])
plt.title('Training and Validation Recall')

plt.subplot(3, 1, 3)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,10000])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()


# In[ ]:


print(history.history)